In [80]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TimeDistributed, Flatten
from tensorflow.keras.layers import LSTM,RepeatVector
from tensorflow.keras.layers import GlobalMaxPooling1D, RepeatVector,Conv1D
import tushare as ts
import pymysql
ts.set_token('b0030b7297c6b297f1db549c7f8c8a080b5f28e2267f07c4741e69ad')

pro = ts.pro_api()
df = pro.daily(ts_code='002594.SZ', start_date='20110630', end_date='20230703')

df = df[['close', 'vol']]
data = df

data.sort_index(ascending=False, inplace=True)
data = data.reset_index(drop=True)

# 计算差值
data_diff = data.diff().dropna()

# 数据缩放
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_diff)

lookback = 30
delay = 1

# 定义生成器函数
def generator(data, lookback=10, delay=1, batch_size=10):
    max_index = len(data) - delay - 1
    i = lookback
    while 1:
        if i + delay >= max_index:
            i = lookback
        samples = np.zeros((batch_size, lookback, data.shape[-1]))
        targets = np.zeros((batch_size, lookback, data.shape[-1]))  # targets should be 3D, same length as inputs
        for j in range(batch_size):
            if i + delay >= max_index:
                i = lookback
            rows = np.arange(i - lookback, i)
            samples[j] = data[rows]
            targets[j] = data[rows + delay]  # targets sequence is shifted by delay
            i += 1
        yield samples, targets

train_gen = generator(data_scaled, lookback=lookback, delay=delay)

# 定义模型
inputs = Input(shape=(lookback, data.shape[-1]))
x = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
x = Dropout(0.5)(x)
x = LSTM(100, return_sequences=True)(x)  # LSTM 层，输出形状为 (batch_size, timesteps, units)

outputs = Dense(data.shape[-1])(x) # 用TimeDistributed包装Dense层，以适应新的目标序列长度
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(), loss='mse')
model.fit(train_gen, epochs=30, steps_per_epoch=20,verbose=0)
    
future = 1
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点
predicted_data = []
for _ in range(future):
    samples = np.reshape(data_to_predict[-lookback:], (1, lookback, data_diff.shape[-1]))
    predictions = model.predict(samples,verbose=0)
    last_prediction = predictions[0, -1, :]
    predicted_data.append(last_prediction)
    # 使用模型的最后一个预测值和前面的实际值（除去最早的一个）来形成新的输入窗口
    data_to_predict = np.vstack([data_to_predict[1:], last_prediction])  

# 将差值转化为原始预测值
predicted_diffs = scaler.inverse_transform(predicted_data)
predicted_data = np.cumsum(predicted_diffs, axis=0) + data.iloc[-lookback-1: -lookback].values

print(round((predicted_data[-1][0]-data.values[-1][0])/data.values[-1][0]*100,2))

2023-07-04 10:45:19.387674: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


-3.25
